In [2]:
import awswrangler as wr
import pandas as pd

In [113]:
pd.set_option('display.max_colwidth', 500)

## Carga de datos

In [3]:
df_hogar_mania = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/hogar_mania')
df_antena3 = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/antena3')
df_saber_vivir = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/saber_vivir')
df_el_mundo = wr.s3.read_parquet(path='s3://raw-recipe-data-bucket/el_mundo')

In [4]:
print(f"Total recetas Hogar Mania: {df_hogar_mania.shape[0]}")
print(f"Total recetas Antena3: {df_antena3.shape[0]}")
print(f"Total recetas Saber Vivir: {df_saber_vivir.shape[0]}")
print(f"Total recetas El Mundo: {df_el_mundo.shape[0]}")
print("----------------")
print(f"Total recetas obtenidas: {df_hogar_mania.shape[0] + df_antena3.shape[0] + df_saber_vivir.shape[0] + df_el_mundo.shape[0]}")

Total recetas Hogar Mania: 7464
Total recetas Antena3: 2022
Total recetas Saber Vivir: 461
Total recetas El Mundo: 388
----------------
Total recetas obtenidas: 10335


In [5]:
df_hogar_mania.columns

Index(['titulo', 'categoria', 'ingredientes', 'elaboracion', 'link'], dtype='object')

In [6]:
df_hogar_mania.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7464 entries, 0 to 7463
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   titulo        7464 non-null   string
 1   categoria     7464 non-null   string
 2   ingredientes  7228 non-null   object
 3   elaboracion   7464 non-null   string
 4   link          7464 non-null   string
dtypes: object(1), string(4)
memory usage: 291.7+ KB


### Análisis

### Hogar Mania

In [10]:
df_hogar_mania.head(5)

,titulo,categoria,ingredientes,elaboracion,link
0,"Buñuelos de viento fáciles, la receta tradicio...",postres,"[125 gr de harina, 30 gr de mantequilla, 1/4 l...",Otoño no es solo época de calabazas y castañas...,https://www.hogarmania.com//cocina/recetas/pos...
1,Corona de hojaldre de Navidad,postres,"[2 láminas de hojaldre rectangular , 150 g de ...","Los polvorones, los turrones, los mazapanes o ...",https://www.hogarmania.com//cocina/recetas/pos...
2,Cafés de Navidad: Gingerbread Latte y Pumpkin ...,postres,"[2 cucharadas de azúcar moreno suave, 1/2 cuch...","La temporada de invierno, junto a la época nav...",https://www.hogarmania.com//cocina/recetas/pos...
3,12 ideas para rellenar el roscón de Reyes,postres,None,"El roscón de Reyes puede comerse solo, aunque ...",https://www.hogarmania.com//cocina/recetas/pos...
4,Mazapanes de Navidad,postres,"[300 gr. de almendra molida , 370 gr. de leche...",Mezcla en un bol la almendra molida con la lec...,https://www.hogarmania.com//cocina/recetas/pos...


In [13]:
df_hogar_mania.isna().sum()

titulo            0
categoria         0
ingredientes    236
elaboracion       0
link              0
dtype: int64

In [143]:
df_hogar_mania[pd.isnull(df_hogar_mania).any(axis=1)]

,titulo,categoria,ingredientes,elaboracion,link,total_ingredientes


In [23]:
df_hogar_mania[pd.isnull(df_hogar_mania).any(axis=1)]['categoria'].value_counts()

categoria
postres                 78
ensaladas-verduras      49
aperitivos              41
pastas-pizzas           12
sopas-cremas            12
bebidas                  9
pescados-mariscos        8
carnes                   6
huevos                   5
arroces-cereales         4
desayunos                4
salsas                   3
cocina-internacional     2
legumbres                1
setas-hongos             1
cocteles                 1
Name: count, dtype: Int64

In [44]:
df_hogar_mania.dropna(inplace=True)

In [8]:
df_hogar_mania["categoria"].value_counts()

categoria
ensaladas-verduras      1457
postres                 1319
carnes                  1293
pescados-mariscos       1121
aperitivos               396
sopas-cremas             378
pastas-pizzas            375
arroces-cereales         333
legumbres                279
huevos                   145
salsas                   106
recetas-pan               72
cocteles                  48
bebidas                   47
setas-hongos              37
cocina-internacional      34
desayunos                 24
Name: count, dtype: Int64

In [64]:
for _, row in df_hogar_mania.head(5).iterrows():
    print(f"{row['titulo']} --- {row['link']}")
    print(f"{row['ingredientes']} --- TOTAL INGREDIENTES {len(row['ingredientes'])}")

Buñuelos de viento fáciles, la receta tradicional de Eva Arguiñano --- https://www.hogarmania.com//cocina/recetas/postres/bunuelos-13530.html
['125 gr de harina' '30 gr de mantequilla' '1/4 l de agua'
 '40 gr de azúcar' '4 huevos' 'ralladura de medio limón' 'sal'
 'aceite de oliva' 'azúcar glas para espolvorear'] --- TOTAL INGREDIENTES 9
Corona de hojaldre de Navidad --- https://www.hogarmania.com//cocina/recetas/postres/corona-navidad-44985.html
['2 láminas de hojaldre rectangular ' '150 g de chocolate negro '
 '40 g de mantequilla ' '1 huevo ' 'azúcar glas '
 '200 g de moras y frambuesas ' 'hojas de menta '] --- TOTAL INGREDIENTES 7
Cafés de Navidad: Gingerbread Latte y Pumpkin Spice Latte --- https://www.hogarmania.com//cocina/recetas/postres/cafes-navidad-gingerbread-pumpkin-spice-latte.html
['2 cucharadas de azúcar moreno suave'
 '1/2 cucharadita de jengibre molido '
 '1/2 cucharadita de nuez moscada molida' '1/2 cucharadita de canela '
 '600 ml. de leche entera'
 '70 ml. café rec

In [61]:
df_hogar_mania['total_ingredientes'] = df_hogar_mania['ingredientes'].apply(len)
df_hogar_mania['total_ingredientes'].value_counts()[:10]

total_ingredientes
11    765
12    740
10    650
9     605
13    597
7     534
14    534
8     517
6     404
15    356
Name: count, dtype: int64

In [63]:
df_hogar_mania['total_ingredientes'].describe()

count    7228.000000
mean       11.054787
std         4.303523
min         0.000000
25%         8.000000
50%        11.000000
75%        14.000000
max        67.000000
Name: total_ingredientes, dtype: float64

In [160]:
df_hogar_mania[df_hogar_mania['total_ingredientes'] == 0].head(5)

,titulo,categoria,ingredientes,elaboracion,link,total_ingredientes
110,Flan de mascarpone y leche condensada,postres,[],"Pon el azúcar en una sartén, agrega unas gotas de zumo de limón y ponlo a calentar hasta que conseguir un caramelo. Cuando empiece a tostarse, remuévelo con una cuchara. Viértelo sobre los moldes y espárcelo bien. Casca los huevos en un bol grande y bátelos con la batidora eléctrica de varillas. Agrega la leche entera, la leche condensada y el queso mascarpone. Sigue batiendo hasta conseguir una crema homogénea. Reparte la mezcla en los moldes especiales para flan. Pon un poco de agua en un ...",https://www.hogarmania.com//cocina/recetas/postres/flan-mascarpone-leche-condensada-24231.html,0
713,Velos de azafrán con macarrón y helado de té,postres,[],"Macarrón de almendra: 500gr de T.P.T90gr Clara de huevo445gr de Merengue italiano T.P.T : 250 gr de Azúcar glace250 gr Almendra en polvo Merengue italiano: 250gr de Azúcar en grano100gr de Agua95 gr de Clara de huevo Mousse de naranja (Relleno del macarrón): 30gr de mascarpone.15gr de mermelada de naranja 90gr de nata1 unid de Gelatina Para la Gelatina de pistilos de azafrán: 250gr de agua0,3gr de hebras de azafrán35gr de azúcar20gr de zumo de limón0,5gr de Agar-Agar Para el helado de té Ea...",https://www.hogarmania.com//cocina/recetas/postres/velos-azafran-macarron-helado-42289.html,0
891,Crepes de chocolate y plátano,postres,[],"Coloca los huevos, la harina, la leche, la mitad de azúcar, un pizca de sal, el aceite y el aroma de vainilla en un vaso batidor. Tritúralos con una batidora hasta que queden perfectamente integrados. Calienta una sartén con una nuez de mantequilla. Vierte una porción de masa, extiéndela bien por toda la sartén y deja que cuaje. Dale la vuelta y deja que cuaje por el otro lado. Repite el proceso hasta terminar toda la masa. Saca los crepes a un plato y resérvalos (el primer crepe siempre se ...",https://www.hogarmania.com//cocina/recetas/postres/crepes-chocolate-platano-26540.html,0
1354,Gazpachuelo tibio de carabineros y rape con almendras,aperitivos,[],"Para el caldo: Cuece todos los ingredientes a fuego lento, espumando en su debido momento, da el punto de sal y cuela sin triturar, tras un cuarto de hora de cocción. Para el gazpachuelo: Dora el puerro picado en el aceite a fuego suave, añade la patata picada finamente y el caldo de pescado, deja que hierva lentamente añadiendo a mitad de la cocción los calabacines (la cocción completa dura unos 20 minutos). En un bol aparte, coloca la mahonesa y añade la preparación anterior poco a poco,...",https://www.hogarmania.com//cocina/recetas/aperitivos/gazpachuelo-tibio-carabineros-rape-almendras-10950.html,0
1433,"Higos a la plancha con riñones, queso del Roncal, tocino crujiente y vinagreta de cacahuetes",aperitivos,[],"Para los riñones: Lamina la cebolla y pica el ajo. Pocha ambos con el aceite a fuego muy suave. Cuando esté blanda la cebolla, y sin tomar mucho color, añade el jerez, el vinagre y el caldo y deja reducir a la mitad de su volumen. Da el punto de sal. Lamina los riñones, sazónalos y pásalos por una plancha caliente con una cucharada de aceite en una sartén, vuelta y vuelta, y reserva al calor. Para los higos a la plancha: Corta los higos en dos mitades. Pon una plancha al fuego con aceite y ...",https://www.hogarmania.com//cocina/recetas/aperitivos/higos-plancha-rinones-queso-roncal-11282.html,0


In [163]:
df_hogar_mania[df_hogar_mania['total_ingredientes'] == 0].shape[0]

18

In [106]:
print(df_hogar_mania.iloc[0]['titulo'].upper())
print("------")
print(df_hogar_mania.iloc[0]["elaboracion"])

BUÑUELOS DE VIENTO FÁCILES, LA RECETA TRADICIONAL DE EVA ARGUIÑANO
------
Otoño no es solo época de calabazas y castañas, también llegan dulces típicos como los buñuelos de viento. Estas esponjosas bolas de masa elaboradas con harina, mantequilla y huevo, fritas en aceite, son un clásico en las pastelerías de España durante esta temporada.
Como los panellets y los huesos de Santo, los buñuelos de viento forman parte de las recetas típicas del Día de Todos los Santos, festividad del 1 de Noviembre. Un postre tradicional muy popular en muchas comunidades autónomas, y también en otras fechas del año. En Cataluña y Baleares se consumen durante la cuaresma como merienda o para acompañar el café, y en Valencia como desayuno. Mientras en Madrid y Andalucía son habituales, sobre todo, en Semana Santa.
La palabra buñuelo tiene dos posibles raíces: romana, el puñuelo, una especie de bola que los romanos amasaban con las manos; y francesa, el beignet, un dulce muy parecido al actual buñuelo. Sin 

#### Antena3

In [115]:
df_antena3.head(5)

,titulo,categoria,ingredientes,elaboracion,link
0,"Receta de alcachofas en salsa, de Karlos Arguiñano: ""Una explosión de sabores""",ensaladas,"[8 huevos de codorniz, 1 cebolla, 2 dientes de ajo, 75 ml de txakoli, 400 ml de caldo de verduras, 1 cucharada de harina, Aceite de oliva virgen extra, Sal, Perejil, Calienta 3-4 cucharadas de aceite en una cazuela. Pela los dientes de ajo, lamínalos y agrégalos. Pela la cebolla, córtala en dados e incorpórala. Sazona y rehoga las hortalizas a fuego suave-medio durante 6-8 minutos. Agrega la harina y rehógala un poco. Vierte el txakoli, mezcla bien y dale un hervor. Vierte el caldo de verdur...","Pon los huevos en una cazuela, cúbrelos con agua y cuécelos durante 5 minutos a partir del momento en que el agua empiece a hervir. Refresca, pela y resérvalos. Calienta agua en la olla rápida y añade unas ramas de perejil. Pela las alcachofas retirándoles 4-5 capas de hojas. Recórtalas por la base y por la parte superior, e introdúcelas en la olla. Sazónalas, cierra la olla y cuécelas durante 4 minutos a partir del momento en que suba la válvula. Retíralas y escúrrelas boca abajo en una fue...",https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/alcachofas-salsa_202405076639f35dc18d400001bd4ea5.html
1,"Una receta sencilla, sabrosa y rápida de Joseba Arguiñano: ensaladilla ibérica",ensaladas,"[3 patatas, 3 zanahorias, 3 huevos, 4 lonchas de queso havarti, 150 g de fuet, 4 lonchas finas de cecina, 4 lonchas finas de jamón ibérico, 8 lonchas finas de lomo ibérico, 150 g de mahonesa, Sal, Picos de pan, Perejil]","Cuece de víspera las patatas con piel (30-35 minutos), las zanahorias peladas (20 minutos) y los huevos (10 minutos). Cuando se templen introdúcelos en el frigorífico y resérvalos para el día siguiente. Pela las patatas y los huevos, y rállalos (con la parte más ancha del rallador) a un bol. Ralla también las zanahorias. Ralla también las zanahorias | antena3.com Corta las lonchas de queso en daditos y añádelo. Retira la piel del fuet, córtalo en cuartos de luna finos e incorpóralo. Agrega...",https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/ensaladilla-iberica_20240506662f629fc0b95c0001d10f67.html
2,"Brócoli con champiñones y patatas, de Arguiñano: ""La receta que levanta el ánimo y ayuda al crecimiento del pelo""",ensaladas,"[6 champiñones, 2 patatas, 4 dientes de ajo, 125 g de jamón serrano (en lochas finas), Aceite de oliva virgen extra, Sal, Perejil]","Calienta agua en una sartén y sazónala. Lava los brócolis, suéltalos en ramilletes, colócalos en el accesorio para cocer al vapor y ponlo sobre la sartén. Tápalo y cuece los brócolis al vapor durante 12-15 minutos. Tápalo y cuece los brócolis al vapor | antena3.com Calienta 3 cucharadas de aceite en una sartén. Pela los ajos, lamínalos, agrégalos a la sartén y rehógalos un poco. Corta las lonchas de jamón en tiras gruesas, incorpóralas y saltéalas brevemente. Lava los champiñones, seca, la...",https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/brocoli-champinones-patatas_20240502663214bcc0b95c0001d6e7be.html
3,"Berenjenas a la crema, de Karlos Arguiñano: ""Una de las 100 mejores recetas del recetario español""",ensaladas,"[1 cebolla pequeña, 4 dientes de ajo, ½ l de leche, 150 ml de caldo de carne, 24 gambas, 60 g de jamón serrano (4 lonchas finas), 60 g de queso gruyere rallado, Aceite de oliva virgen extra, Harina, Sal, Perejil]","Calienta abundante aceite en una sartén grande, pela los ajos, añádelos (enteros) y fríelos a fuego medio hasta que se doren. Retíralos y tíralos. Pela las berenjenas, córtalas en rodajas de 1 cm, extiéndelas en una fuente y sazónalas. Pásalas por harina, introdúcelas en la sartén (a tandas) y fríelas por los 2 lados. Retíralas y escúrrelas sobre una fuente cubierta con papel absorbente de cocina. Pásalas por harina, introdúcelas en la sartén | antena3.com Cubre la base de 4 cazuelitas ind...",https://www.antena3.com/programas/karlos-arguinano/recetas/

In [125]:
df_antena3.isna().sum()

titulo                0
categoria             0
ingredientes          0
elaboracion           0
link                  0
total_ingredientes    0
dtype: int64

In [121]:
df_antena3["categoria"].value_counts()

categoria
ensaladas    494
pescado      336
carnes       317
postres      317
huevos       205
entrantes    130
pasta        113
guisos        80
arroces       30
Name: count, dtype: Int64

In [122]:
df_antena3['total_ingredientes'] = df_antena3['ingredientes'].apply(len)
df_antena3['total_ingredientes'].value_counts()[:10]

total_ingredientes
11    185
10    174
8     174
7     173
9     161
12    160
13    127
14    103
0      98
15     84
Name: count, dtype: int64

In [138]:
df_antena3['total_ingredientes'].describe()

count    2022.000000
mean       12.241345
std         8.403253
min         0.000000
25%         8.000000
50%        11.000000
75%        15.000000
max       137.000000
Name: total_ingredientes, dtype: float64

In [129]:
df_antena3[df_antena3['total_ingredientes'] == 0]['categoria'].value_counts()

categoria
huevos       23
ensaladas    21
postres      20
pasta        11
pescado       9
carnes        9
entrantes     3
guisos        2
Name: count, dtype: Int64

In [144]:
df_antena3[df_antena3['total_ingredientes'] == 0]

,titulo,categoria,ingredientes,elaboracion,link,total_ingredientes
5,"Karlos Arguiñano: ""Esta es la forma más sencilla de hacer alcachofas""",ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/forma-mas-sencilla-hacer-alcachofas_202404216620c75cc0b95c0001023a3c.html,0
69,"La ensalada exótica de Arguiñano: de escarola, fresas, aguacate y un aliño especial",ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/escarola-fresas-aguacate-alino_20230524646df87c910a1b0001d95a8a.html,0
77,"Karlos Arguiñano sorprende con una ensalada de escarola, fresa y aguacate",ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/ensalada-escarola-fresa-aguacate_2023032964242122a1e0b90001dd3ccd.html,0
130,Karlos Arguiñano: Las formas más variadas y saludables de elaborar unos cogollos refrescantes,ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/variadas-saludables-elaborar-cogollos_2022071462cff57b98b3200001cfc1ef.html,0
158,"¿Te apasionan las berenjenas? Disfruta de las recetas más ricas, rápidas y fáciles de Arguiñano",ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/apasionan-berenjenas-disfruta-recetas-mas-ricas-rapidas-faciles_202203296242cb5043a63f0001cc438f.html,0
...,...,...,...,...,...,...
1974,El juego por el Día de la Lasaña: ¿Cuál ha sido la receta de lasaña más vista de Karlos Arguiñano?,pasta,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/pasta/juego-dia-lasana-cual-sido-receta-mas-vista_202007295f21431b6cf6da000157a0e1.html,0
1990,"Cazuela de arroz, achicoria y huevo cocido",pasta,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/pasta/cazuela-de-arroz-achicoria-y-huevo-cocido_202001225e283cca0cf2ac911f7577b2.html,0
1999,Receta de Fideos con verduras y pollo,pasta,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/pasta/receta-de-fideos-con-verduras-y-pollo_201909305d91eda40cf2ebc7f5a3b176.html,0
2000,Receta de Macarrones con compota de tomate y champiñones,pasta,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/pasta/receta-de-macarrones-con-compota-de-tomate-y-champinones_201909245d89f69a0cf20dcdcc48908c.html,0


In [153]:
df_antena3[(df_antena3['elaboracion'].str.len() == 0) & (df_antena3['total_ingredientes'] == 0)]

,titulo,categoria,ingredientes,elaboracion,link,total_ingredientes
5,"Karlos Arguiñano: ""Esta es la forma más sencilla de hacer alcachofas""",ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/forma-mas-sencilla-hacer-alcachofas_202404216620c75cc0b95c0001023a3c.html,0
69,"La ensalada exótica de Arguiñano: de escarola, fresas, aguacate y un aliño especial",ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/escarola-fresas-aguacate-alino_20230524646df87c910a1b0001d95a8a.html,0
77,"Karlos Arguiñano sorprende con una ensalada de escarola, fresa y aguacate",ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/ensalada-escarola-fresa-aguacate_2023032964242122a1e0b90001dd3ccd.html,0
130,Karlos Arguiñano: Las formas más variadas y saludables de elaborar unos cogollos refrescantes,ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/variadas-saludables-elaborar-cogollos_2022071462cff57b98b3200001cfc1ef.html,0
158,"¿Te apasionan las berenjenas? Disfruta de las recetas más ricas, rápidas y fáciles de Arguiñano",ensaladas,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/apasionan-berenjenas-disfruta-recetas-mas-ricas-rapidas-faciles_202203296242cb5043a63f0001cc438f.html,0
...,...,...,...,...,...,...
1974,El juego por el Día de la Lasaña: ¿Cuál ha sido la receta de lasaña más vista de Karlos Arguiñano?,pasta,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/pasta/juego-dia-lasana-cual-sido-receta-mas-vista_202007295f21431b6cf6da000157a0e1.html,0
1990,"Cazuela de arroz, achicoria y huevo cocido",pasta,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/pasta/cazuela-de-arroz-achicoria-y-huevo-cocido_202001225e283cca0cf2ac911f7577b2.html,0
1999,Receta de Fideos con verduras y pollo,pasta,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/pasta/receta-de-fideos-con-verduras-y-pollo_201909305d91eda40cf2ebc7f5a3b176.html,0
2000,Receta de Macarrones con compota de tomate y champiñones,pasta,[],,https://www.antena3.com/programas/karlos-arguinano/recetas/pasta/receta-de-macarrones-con-compota-de-tomate-y-champinones_201909245d89f69a0cf20dcdcc48908c.html,0


In [154]:
df_antena3[(df_antena3['elaboracion'].str.len() == 0) & (df_antena3['total_ingredientes'] > 0)]

,titulo,categoria,ingredientes,elaboracion,link,total_ingredientes
14,"Arguiñano, sobre la ensalada de endibias asadas y cremoso de gorgonzola: ""¡Qué ganas tengo de hincarle el diente!""",ensaladas,"[8 endibias, Sal, 40 g de mantequilla, Para las salsas:, 225 g de queso gorgonzola, 100 ml de aceite de girasol aromatizado con ajo, 150 g de nueces peladas, 150 ml de aceite de oliva virgen extra, Sal, Perejil, Pon el queso gorgonzola en un vaso americano, vierte la mitad del aceite aromatizado con ajo y tritura bien con una batidora eléctrica. Vierte el resto del aceite (ajos incluidos) y sigue triturando hasta que los ingredientes queden bien integrados., Calienta una sartén con la mitad ...",,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/ensalada-endibias-asadas-cremoso-gorgonzola_2024030465e5a59129d8140001716cf6.html,15
15,"Receta de cogollos de lechuga a la plancha con atún, de Karlos Arguiñano",ensaladas,"[6 cogollos de lechuga, 200 g de atún en aceite, 1 cebolla roja pequeña, 10 g de pistachos pelados, 10 g de avellanas peladas, 1 cucharada de arándanos deshidratados, 1 cucharada de miel, Aceite de oliva virgen extra, Vinagre de manzana, Sal, Perejil, Para hacer la vinagreta, mezcla en un bol la miel con 2 cucharadas de vinagre y 100 ml de aceite. Bate los ingredientes con una varilla de mano., Pica los pistachos, las avellanas y los arándanos, y agrégalos al bol. Remueve los ingredientes y ...",,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/cogollos-lechuga-plancha-atun_2024022665dc7690344c980001c04730.html,16
16,"Ensalada de repollo y zanahoria con manzana y mostaza, de Arguiñano",ensaladas,"[½ repollo morado, 3 zanahorias, 1 manzana Granny Smith, Zumo de 1 limón, 2 cucharadas de vinagre de manzana, 1 cucharada de semillas de hinojo, 2 cucharadas de mostaza de Dijon, 150 g de mahonesa, Sal, Perejil, Pon el zumo del limón en un bol grande. Añade el vinagre, las semillas de hinojo, la mostaza y la mahonesa. Mezcla los ingredientes y remuévelos con una varilla manual hasta que queden bien integrados. Reserva el aliño., Corta los repollos en juliana fina y ponlos en un bol grande., ...",,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/ensalada-repollo-zanahoria-manzana-mostaza_2024022265d739514129260001ceec3b.html,15
101,Un entrante de gala de Arguiñano: endibias con queso azul y nueces,ensaladas,"[2-3 endibias, 150 g de queso azul, 12 nueces peladas, 6 orejones de albaricoque, 1 pera, 1 cucharada de miel, Aceite de oliva virgen extra, vinagre, perejil, Pon la miel en un bote de cristal, vierte 1 cucharada de vinagre y 2-3 cucharadas de aceite. Tapa el bote y bate los ingredientes. Reserva el aliño., Corta la base de las endibias, separa las hojas exteriores (las más grandes) y ponlas en una fuente. Pica finamente las hojas centrales y ponlas en un bol. Pica los orejones en daditos y ...",,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/endibias-queso-azul-nueces_2022122263a4416e3a2bb40001322545.html,12
106,¡Boniato a todas horas! El protagonista en las recetas de invierno de Karlos Arguiñano,ensaladas,[Recuerda que las cremas son una buena opción para introducir en la dieta de los más pequeños un sinfín de hortalizas que de otra manera se negarían a comer.],,https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/boniato-protagonista-invierno_2022111763762351bbdbfb0001b0a32a.html,1
...,...,...,...,...,...,...
1950,"Receta de fideos marineros, de Karlos Arguiñano",pasta,"[225 g de fideo nº 2, 1 calamar mediano limpio, 1 rape mediano limpio (con huesos y cabeza), 12 almejas, 12 mejillones, 1 pimiento verde, ½ pimiento rojo, 3 dientes de ajo, 1 ñora, 1 huevo, aceite de oliva virgen extra, vinagre, sal, 1 cucharadita de pimentón, 10 hebras de azafrán, perejil, Calienta abundante agua en una cazuela. Agrega los huesos y cabeza del rape, unas ramas de perejil y una pizca de sal. Cuece los ingredientes durante 15 minutos (retira la

In [158]:
for _, row in df_antena3.head(3).iterrows():
    print(f"{row['titulo']} \n--- {row['link']}")
    print(f"{row['ingredientes']} \n--- TOTAL INGREDIENTES {len(row['ingredientes'])}")

Receta de alcachofas en salsa, de Karlos Arguiñano: "Una explosión de sabores" 
--- https://www.antena3.com/programas/karlos-arguinano/recetas/ensaladas/alcachofas-salsa_202405076639f35dc18d400001bd4ea5.html
['8 huevos de codorniz' '1 cebolla' '2 dientes de ajo' '75 ml de txakoli'
 '400 ml de caldo de verduras' '1 cucharada de harina'
 'Aceite de oliva virgen extra' 'Sal' 'Perejil'
 'Calienta 3-4 cucharadas de aceite en una cazuela. Pela los dientes de ajo, lamínalos y agrégalos. Pela la cebolla, córtala en dados e incorpórala. Sazona y rehoga las hortalizas a fuego suave-medio durante 6-8 minutos. Agrega la harina y rehógala un poco. Vierte el txakoli, mezcla bien y dale un hervor. Vierte el caldo de verdura y cocina la salsa a fuego suave durante 10 minutos y pásala a una jarra. Tritura los ingredientes con una batidora eléctrica hasta conseguir una salsa homogénea.'
 'Pon las alcachofas en una tartera (cazuela amplia y baja), viérteles la salsa por encima y cocínalas en la salsa dur

#### Saber Vivir

#### El Mundo